## Multi-Accent and Multi-Lingual Voice Clone Demo with MeloTTS

In [15]:
import os
os.environ['TRANSFORMERS_VERBOSITY'] = 'info'
os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'
os.environ['CURL_CA_BUNDLE'] = ''
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

In [16]:
import certifi

print(certifi.where())

C:\Users\RSairam\Documents\MyRepo\EchoQuestPython\echoquestpythoncloning2\.cleanvenv\lib\site-packages\certifi\cacert.pem


### Initialization

In this example, we will use the checkpoints from OpenVoiceV2. OpenVoiceV2 is trained with more aggressive augmentations and thus demonstrate better robustness in some cases.

In [23]:
import sys
import os
from pathlib import Path
BASE_DIR = Path("C:/Users/RSairam/Documents/MyRepo/EchoQuestPython/echoquestpythoncloning2")
CHECKPOINT_DIR = BASE_DIR /"checkpoints_v2"
BASE_SPEAKER_DIR = CHECKPOINT_DIR / "base_speakers" / "ses"
CONFIG_PATH = CHECKPOINT_DIR / "converter" / "config.json"
CKPT_PATH = CHECKPOINT_DIR / "converter" / "checkpoint.pth"
ckpt_converter = 'checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(str(CONFIG_PATH), device=device)
tone_color_converter.load_ckpt(str(CKPT_PATH))

os.makedirs(output_dir, exist_ok=True)

Loaded checkpoint 'C:\Users\RSairam\Documents\MyRepo\EchoQuestPython\echoquestpythoncloning2\checkpoints_v2\converter\checkpoint.pth'
missing/unexpected keys: [] []


### Obtain Tone Color Embedding
We only extract the tone color embedding for the target speaker. The source tone color embeddings can be directly loaded from `checkpoints_v2/ses` folder.

In [25]:

reference_speaker = 'C:/Users/RSairam/Downloads/AUD-20250705-WA0003.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=True)

OpenVoice version: v2
[(12.238, 13.458)]
after vad: dur = 1.22


AssertionError: input audio is too short

In [13]:
pip install requests==2.27.1

  Obtaining dependency information for requests==2.27.1 from https://files.pythonhosted.org/packages/2d/61/08076519c80041bc0ffa1a8af0cbd3bf3e2b62af10435d269a9d0f40564d/requests-2.27.1-py2.py3-none-any.whl.metadata
  Using cached requests-2.27.1-py2.py3-none-any.whl.metadata (5.0 kB)
Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-server 2.27.3 requires requests>=2.31, but you have requests 2.27.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
pip install --upgrade certifi

Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for certifi from https://files.pythonhosted.org/packages/4f/52/34c6cf5bb9285074dc3531c437b3919e825d976fde097a7a73f79e726d03/certifi-2025.7.14-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/162.7 kB ? eta -:--:--
   ------------------------------ --------- 122.9/162.7 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 162.7/162.7 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: certifi
    Found existing installation: certifi 2025.4.26
    Uninstalling certifi-2025.4.26:
      Successfully uninstalled certifi-2025.4.26



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
python -m certifi


SyntaxError: invalid syntax (997592317.py, line 1)

#### Use MeloTTS as Base Speakers

MeloTTS is a high-quality multi-lingual text-to-speech library by @MyShell.ai, supporting languages including English (American, British, Indian, Australian, Default), Spanish, French, Chinese, Japanese, Korean. In the following example, we will use the models in MeloTTS as the base speakers. 

from melo.api import TTS


texts = {
    'EN_NEWEST': "Did you ever hear a folk tale about a giant turtle?",  # The newest English base speaker model
    'EN': "Did you ever hear a folk tale about a giant turtle?",
    'ES': "El resplandor del sol acaricia las olas, pintando el cielo con una paleta deslumbrante.",
    'FR': "La lueur dorée du soleil caresse les vagues, peignant le ciel d'une palette éblouissante.",
    'ZH': "在这次vacation中，我们计划去Paris欣赏埃菲尔铁塔和卢浮宫的美景。",
    'JP': "彼は毎朝ジョギングをして体を健康に保っています。",
    'KR': "안녕하세요! 오늘은 날씨가 정말 좋네요.",
}
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

src_path = f'{output_dir}/tmp.wav'

# Speed is adjustable
speed = 1.0

for language, text in texts.items():
    model = TTS(language=language, device=device)
    speaker_ids = model.hps.data.spk2id
    
    for speaker_key in speaker_ids.keys():
        speaker_id = speaker_ids[speaker_key]
        speaker_key = speaker_key.lower().replace('_', '-')
        source_se_path = BASE_SPEAKER_DIR / f"{speaker_key}.pth"
        #source_se = torch.load(f'checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
        source_se = torch.load(source_se_path, map_location=DEVICE)
        if torch.backends.mps.is_available() and device == 'cpu':
            torch.backends.mps.is_available = lambda: False
        model.tts_to_file(text, speaker_id, src_path, speed=speed)
        save_path = f'{output_dir}/output_v2_{speaker_key}.wav'

        # Run the tone color converter
        encode_message = "@MyShell"
        tone_color_converter.convert(
            audio_src_path=src_path, 
            src_se=source_se, 
            tgt_se=target_se, 
            output_path=save_path,
            message=encode_message)